In [1]:
from osgeo import gdal
import numpy as np
import s3fs
import os
import os.path
import netCDF4

In [2]:
# Adjust the variable inputs to create the prefix for the file of interest
bucket_name = 'noaa-goes16' # Change to 'noaa-goes17' for GOES-17 satellite
product_name = 'ABI-L2-ACHAC'# See 'https://docs.opendata.aws/noaa-goes16/cics-readme.html' for other GOES-R Series products
year = 2020 
day_of_year = 296 
hour = 16

# Adjust pathname to local directory
directory = "/Users/daniellelosos/"

In [3]:
# Make new directories to store files
if not os.path.isdir(product_name +"_netCDFs"):
    os.mkdir(product_name +"_netCDFs")
if not os.path.isdir(product_name +"_GeoTIFFs"):
    os.mkdir(product_name +"_GeoTIFFs") 

In [4]:
# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Use anonymous credentials to access public data  from AWS
fs = s3fs.S3FileSystem(anon=True)

# Write prefix for the files of interest, and list all files beginning with this prefix.
prefix = f'{bucket_name}/{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/'
print(prefix)
files = fs.ls(prefix)

noaa-goes16/ABI-L2-ACHAC/2020/296/16/


In [5]:
#Read the first netCDF file in the list (closest to the top of the hour) and download
first_file = files[0]
path = "/Users/daniellelosos/"+ product_name +"_netCDFs/" + first_file.split("/")[-1]
fs.download(first_file, path)
dataset = gdal.Open(path, gdal.GA_ReadOnly)
print ("DATASET:\n" + first_file)
print ("\nSUBDATASETS:")
subdatasets = dataset.GetSubDatasets()
for fname, name in subdatasets:
    print (name)
    print (fname)

DATASET:
noaa-goes16/ABI-L2-ACHAC/2020/296/16/OR_ABI-L2-ACHAC-M6_G16_s20202961601166_e20202961603539_c20202961605266.nc

SUBDATASETS:
[300x500] geopotential_height_at_cloud_top (16-bit integer)
NETCDF:"/Users/daniellelosos/ABI-L2-ACHAC_netCDFs/OR_ABI-L2-ACHAC-M6_G16_s20202961601166_e20202961603539_c20202961605266.nc":HT
[300x500] status_flag (8-bit integer)
NETCDF:"/Users/daniellelosos/ABI-L2-ACHAC_netCDFs/OR_ABI-L2-ACHAC-M6_G16_s20202961601166_e20202961603539_c20202961605266.nc":DQF


In [6]:
# Choose the first variable/subdataset in the netCDF file to convert to a GeoTIFF.
# See the file's other variables in the "SUBDATASETS:" output, formatted as NETCDF:"filename":variable
# If the first subdataset does not contain the variable of intrest, adjust the list number below
f = netCDF4.Dataset(path)
var = list(f.variables.keys())[0] # (Optional) Adjust this list number, ie. [0] selects the 1st subdataset 
print ("Selected variable: " + var)

Selected variable: HT


In [7]:
# Convert netCDF file to geoTiff
print ("\nConversion in progress...")
layer = gdal.Open("NETCDF:{0}:{1}".format(path, var))
tiff_name = product_name + "_" + var + ".tif"
geotiff_pathname = os.path.join(directory + product_name +"_GeoTiffs", tiff_name)
geotiff = gdal.Translate(geotiff_pathname, layer, format = "NetCDF")
print("\n" + tiff_name + " has been generated")


Conversion in progress...

ABI-L2-ACHAC_HT.tif has been generated
